In [4]:
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np
import re
from urllib import urlopen
html = urlopen("https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans")
bsObj = BeautifulSoup(html.read(),"lxml")

csvFile = open("myComments_V5.csv",'wt')
writer = csv.writer(csvFile)
page = 0
users = {}

try:
    while (bsObj.find("a",{"class":"Next"})!=None):
        comments = bsObj.findAll("div",{"class":"Comment"})
        for i in comments:
            text = unicode(i.find("div",{"class":"Message"}).get_text()).encode('utf-8').strip()
            # Some reviews starting with "xxx@:" are replies to previous posts, 
            # so we need to exclude the original message to avoid double-count
            if text.split("\n",1)[0].find('said:')>=1:
                text = text.split("\n",1)[1:]
            user = unicode(i.find("a").attrs["title"]).encode('utf-8')
            # Filter posts below 120 characters
            if len(text)>120:
                if user not in users.keys():
                    users[user] = 1
                    date = unicode(i.find("time").attrs["title"]).encode('utf-8')
                    writer.writerow([date,user,text])
                else:
                    # Several active users left more than 1000 reviews and may seriously bias our analysis
                    if users[user] <=30:
                        users[user] += 1
                        date = unicode(i.find("time").attrs["title"]).encode('utf-8')
                        writer.writerow([date,user,text])                    
        nextHTML = bsObj.find("a",{"class":"Next"})["href"]
        html = urlopen(nextHTML)
        bsObj = BeautifulSoup(html.read(),"lxml")
        page = page+1
        
finally:
    csvFile.close()